<a href="https://colab.research.google.com/github/PabloLBandeira/biblioteca-Pandas/blob/main/recomendacao_trending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.escoladnc.com.br/wp-content/uploads/2022/06/dnc_formacao_dados_logo_principal_preto-1.svg" alt="drawing" width="300"/>

# Recomendação Trending

<!-- Este notebook contém 2 exemplos de recomendação Top N:

- **Top N consumidos**: os N itens mais consumidos pelos usuários
- **Top N avaliados**: os N itens melhores avaliados pelos usuários -->

O dataset a ser utilizado será o [MovieLens](https://grouplens.org/datasets/movielens/), cuja análise exploratória foi feita no exemplo prático do módulo **Introdução aos Sistemas de Recomendação**.

In [1]:
import os
import re
import sys
import pandas as pd
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

# Carregando e processando o dataset

Para mais informações desta sessão, consulte o notebook `Introdução aos Sistemas de Recomendação` do módulo 01.

## Arquivo de avaliações

Upload file `ratings.parquet`

In [ ]:
%%time
_ = files.upload()

In [ ]:
def convert_timestamp_to_date(timestamp:int):
    return datetime.fromtimestamp(timestamp).date()

df_ratings = pd.read_parquet('ratings.parquet')
df_ratings['date'] = df_ratings['timestamp'].apply(convert_timestamp_to_date)
df_ratings.tail()

## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [ ]:
%%time
_ = files.upload()

In [ ]:
def extract_year_from_title(title:str, regex='(\d{4})'):
    match = re.search(regex, title)
    return None if match is None else match.group()

def convert_genres_to_list(genres:str, separator='|'):
    return genres.split(separator)

df_items = pd.read_parquet('movies.parquet')
df_items['genres'] = df_items['genres'].apply(convert_genres_to_list)
df_items['year'] = df_items['title'].apply(extract_year_from_title)
df_items.tail()

# Cálculo do Trending

A recomendação _trending_ busca apresentar os itens que tiveram maior _lift_ de consumo em uma determinada janela de tempo. Em termos matemáticos, o _lift_ pode ser definido como:

$$lift = \frac{consumoJanelaAtual-consumoJanelaAnterior}{consumoJanelaAnterior}$$

Neste notebook, utilizaremos como janela de tempo o mês de consumo do item. A função abaixo nos auxilia a definir uma coluna `window` que será utilizada na lógica do algoritmo.

In [ ]:
def extract_year_month(date):
    return '{:04d}-{:02d}'.format(date.year, date.month)

df_ratings['window'] = df_ratings['date'].apply(extract_year_month)
df_ratings.tail()

## Consumo por janela temporal

In [ ]:
df_window_consumptions = (
    df_ratings
    .groupby(['item_id', 'window'])
    .agg({'user_id': 'count'})
    .reset_index()
    .rename({'user_id': 'count'}, axis=1)
    .sort_values(by=['item_id', 'window'])
)
df_window_consumptions

## Shift temporal

Para realizar operações entre o valor atual e o anterior de uma janela temporal podemos **deslocar** (_shift_) os valores de uma coluna a partir de um agrupamento. Exemplo:

| Agrupamento | Valor | Shift Valor |
|--------|-------|-------------|
| A      | A1    | N/A         |
| A      | A2    | A1          |
| B      | B1    | N/A         |
| B      | B2    | B1          |
| B      | B3    | B2          |

In [ ]:
df_window_consumptions.sort_values(by=['item_id', 'window'], inplace=True)

df_window_consumptions['count_previous'] = (
    df_window_consumptions
    .groupby(['item_id'])['count']
    .shift(1)
)
df_window_consumptions

## Lift

Implementando a seguinte fórmula:
$$lift = \frac{contagemJanelaAtual-contagemJanelaAnterior}{contagemJanelaAnterior}$$

In [ ]:
df_window_consumptions['lift'] = (df_window_consumptions['count'] - df_window_consumptions['count_previous'])/df_window_consumptions['count_previous']
df_window_consumptions

## Especificando janela de recomendação

Para a recomendação precisamos de uma janela de referência. Por exemplo:

- Trending na janela atual?
- Trending na janela anterior?
- Trending em uma janela específica?

Definida a janela de referência, utilizamos o valor do _lift_ como _score_ do item e ordenamos pelo _score_.

In [ ]:
prediction_window = '2003-01'
(
    df_window_consumptions
    .query('window == @prediction_window')
    .rename({'lift': 'score'}, axis=1)
    .sort_values(by='score', ascending=False)
)

_____________

# Recomendando Itens Trending

Por fim, juntamos toda a lógica descrita até então na função `recommend_trending_n` abaixo para recomendar os N items que estão em maior ascensão.

In [ ]:
def recommend_trending_n(ratings:pd.DataFrame, n:int, prediction_window:str=None, min_evaluations:int=None) -> pd.DataFrame:

    prediction_window = max(ratings['window']) if prediction_window is None else prediction_window

    ratings = ratings[['item_id', 'window', 'user_id']]
    # Calculo de janela
    df_window_consumptions = (
        ratings
        .groupby(['item_id', 'window'])['user_id']
        .count()
        .reset_index()
        .rename({'user_id': 'count'}, axis=1)
        .sort_values(by=['item_id', 'window'])
    )

    # Shift temporal
    df_window_consumptions['count_previous'] = (
        df_window_consumptions
        .groupby(['item_id'])['count']
        .shift(1)
    )

    # Calculo do lift
    df_window_consumptions['lift'] = (df_window_consumptions['count'] - df_window_consumptions['count_previous'])/df_window_consumptions['count_previous']

    # Selecao de janela
    recommendations = (
      df_window_consumptions
      .query('window == @prediction_window')
      .rename({'lift': 'score'}, axis=1)
      .sort_values(by='score', ascending=False)
    )

    if min_evaluations is not None:
        recommendations = recommendations.query('count_previous >= @min_evaluations')

    return recommendations.head(n)

df_trending = recommend_trending_n(df_ratings, n=10, prediction_window='2002-12')
df_trending.merge(df_items, on='item_id', how='inner')

Note que alguns itens tiveram _lift_/_score_ altos, porém **poucos usuários consumiram estes itens**.

Para evitar estes comportamentos, podemos estabelecer um limite mínimo de consumo para que um item seja recomendável.

In [ ]:
prediction_window = '2002-12'
min_evaluations = 2
n = 10

df_trending = recommend_trending_n(
    df_ratings,
    n=n,
    prediction_window=prediction_window,
    min_evaluations=min_evaluations
)
df_trending.merge(df_items, on='item_id', how='inner')

# Seleção de Acervo

Podemos selecionar itens específicos a serem recomendados filtrando o dataset de avaliações.

In [ ]:
genre = "Children's"
item_ids = df_items[df_items['genres'].apply(lambda x: genre in x)]['item_id']
df_ratings_filtered = df_ratings[df_ratings['item_id'].isin(item_ids)]
df_ratings_filtered.tail()

In [ ]:
df_trending = recommend_trending_n(df_ratings_filtered, n=10, prediction_window='2002-12', min_evaluations=2)
df_trending.merge(df_items, on='item_id', how='inner')